In [3]:
#D'abord vérifier si on a du Lenet

In [33]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model, preprocessing 
from keras.applications.imagenet_utils import preprocess_input
from sklearn import metrics
%matplotlib inline


In [34]:
def build_model(width, height, depth, classes):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(16, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 3 => POOL layer set
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(256),
		Activation("relu"),
		BatchNormalization(),
		Dropout(0.5),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [35]:
#Essayons de faire lenet avec tf voir ci-dessous . Il y a aussi une commande pour voir le résumer peut être .resume
def build_model(width, height, depth, classes,drop_out_rate):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(30, (5, 5), padding="same", input_shape=inputShape),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
        
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(16, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),

        
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(128),
		Activation("relu"),
		BatchNormalization(),
		Dropout(drop_out_rate),
        
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [36]:

df=pd.read_csv("/home/marcpozzo/Desktop/c3po/Images_aquises/generateur.csv")
df=df[(df["class"]=='corneille') | (df["class"]=='pigeon') | (df["class"]=='faisan')   ]
df.drop('labels',inplace=True,axis=1)
df.head()

,class,img_paths
2,corneille,Rec_images/DSCF0180_corneille_2.JPG
3,corneille,Rec_images/DSCF0180_corneille_3.JPG
4,pigeon,Rec_images/DSCF0258_pigeon_4.JPG
8,pigeon,Rec_images/DSCF0335_pigeon_8.JPG
10,pigeon,Rec_images/DSCF0341_pigeon_10.JPG


In [37]:
#Paramètres

#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size=0.2
epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip=True

In [38]:

Minimum_Number_Class=100
dropout_rate=0.3
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1

In [39]:

base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'

df=pd.read_csv(generateur_path)
df.drop('labels',inplace=True,axis=1)

df["class"].unique()
for c in df:
    print(''+c+':',len(df[c].unique()))
df["class"].unique()


All_Unique=df["class"].unique()
Utilisable=[]
for i in df["class"].unique():
    if df["class"][df["class"]==i].count()>Minimum_Number_Class:
        Utilisable.append(i)
Utilisable
Non_Utilisable=set(All_Unique)-set(Utilisable)
Non_Utilisable
for i in Non_Utilisable:
    df=df[df["class"]!=i]
df=df[df["class"]!="oiseau"]  
df["class"].unique()


for i in range(len(df["class"])):
    image_name=df["img_paths"].iloc[i]
    df["img_paths"].iloc[i]=os.path.join(base_img_paths,image_name)
    

class: 13
img_paths: 6220


In [40]:
n_path="/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_images/"



fp_new=pd.read_csv("/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_threshold/table_fp_new.csv")

liste_img_paths=[]
for i in range(len(fp_new)):
    liste_img_paths.append(n_path+fp_new["imagetteName"].iloc[i])

fp_new["img_paths"]=liste_img_paths
#imagetteNamefp_new.head()

fp_new["class"]="autre"
fp_new.columns

to_drop=['path', 'filename', 'imagetteName', 'max_cat', 'cat', 'xmin', 'xmax','ymin', 'ymax', 'former_index']

fp_new.drop(to_drop,axis=1,inplace=True)

df=pd.concat([df,fp_new])

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [42]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 5,
        zoom_range = [0.9,1.3],
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip = horizontal_flip)





test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))

gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]


Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


In [ ]:
#First test the influence of number of epochs on the other recall

In [2]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 5,
        zoom_range = 1.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip = horizontal_flip)





test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))
"""
gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]
"""

Using TensorFlow backend.


NameError: name 'tf' is not defined

In [17]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(train_generator,epochs=100,validation_data=test_generator,verbose=0)
model.save("tf_fp_100ep_gen1")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tf_fp_100ep_gen1/assets


In [18]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(train_generator,epochs=200,validation_data=test_generator,verbose=0)
model.save("tf_fp_200ep_gen1")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
INFO:tensorflow:Assets written to: tf_fp_200ep_gen1/assets


In [19]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit_generator(train_generator,epochs=100,validation_data=test_generator,verbose=0)
model.save("tf_fp_100ep_gen2")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
INFO:tensorflow:Assets written to: tf_fp_100ep_gen2/assets


In [20]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit_generator(train_generator,epochs=200,validation_data=test_generator,verbose=0)
model.save("tf_fp_200ep_gen2")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
INFO:tensorflow:Assets written to: tf_fp_200ep_gen2/assets


In [17]:
#Serie

In [25]:
2

In [12]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 0,
        zoom_range = 1.1,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip = False)





test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))
"""
gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]
"""

y_train=train_generator.classes
y_test=test_generator.classes

Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


'\ngen=train_generator[0]\n\nx_train=gen[0]\ny_train=gen[1]\n\n\n\ngen_test=test_generator[0]\n\nx_test=gen_test[0]\ny_test=gen_test[1]\n'

In [14]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(train_generator,epochs=100,validation_data=test_generator,verbose=0)
#model.save("tf_fp_200ep_gen1")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

y_predict=model.predict(test_generator).argmax(axis=1)

In [15]:
print(metrics.classification_report(y_predict,y_test,digits=5))

              precision    recall  f1-score   support

           0    0.93328   0.95013   0.94163      9525
           1    0.00000   0.00000   0.00000         0
           2    0.06047   0.02664   0.03698       488
           3    0.00000   0.00000   0.00000         0
           4    0.00000   0.00000   0.00000         1
           5    0.01807   0.01554   0.01671       193

    accuracy                        0.88821     10207
   macro avg    0.16864   0.16539   0.16589     10207
weighted avg    0.87415   0.88821   0.88080     10207



In [16]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        #preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 0,
        zoom_range = 1.1,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip = False)





test_data_generator = ImageDataGenerator()




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))
"""
gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]
"""

y_train=train_generator.classes
y_test=test_generator.classes


#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(train_generator,epochs=100,validation_data=test_generator,verbose=0)
#model.save("tf_fp_200ep_gen1")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

y_predict=model.predict(test_generator).argmax(axis=1)

print(metrics.classification_report(y_predict,y_test,digits=5))

Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
              precision    recall  f1-score   support

           0    0.98989   0.95011   0.96960     10103
           1    0.00000   0.00000   0.00000         0
           2    0.02326   0.05495   0.03268        91
           3    0.00000   0.00000   0.00000         1
           4    0.00000   0.00000   0.00000         5
           5    0.00602   0.14286   0.01156         7

    accuracy                        0.94102     10207
   macro avg    0.16986   0.19132   0.16897     10207
weighted avg    0.98002   0.94102   0.96002     10207



/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        #preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 0,
        #zoom_range = 1.1,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip = False)





test_data_generator = ImageDataGenerator()




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))
"""
gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]
"""

y_train=train_generator.classes
y_test=test_generator.classes


#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(train_generator,epochs=100,validation_data=test_generator,verbose=0)
#model.save("tf_fp_200ep_gen1")
#y_predict=model.predict(x_test).argmax(axis=1)
#print(metrics.classification_report(y_predict,y_test,digits=5))

y_predict=model.predict(test_generator).argmax(axis=1)

print(metrics.classification_report(y_predict,y_test,digits=5))

Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
              precision    recall  f1-score   support

           0    0.99247   0.95005   0.97080     10130
           1    0.00000   0.00000   0.00000         2
           2    0.00930   0.03279   0.01449        61
           3    0.00000   0.00000   0.00000         5
           4    0.00000   0.00000   0.00000         2
           5    0.00000   0.00000   0.00000         7

    accuracy                        0.94308     10207
   macro avg    0.16696   0.16381   0.16422     10207
weighted avg    0.98504   0.94308   0.96356     10207



In [22]:
y_predict=model.predict_generator(test_generator).argmax(axis=1)

print(metrics.classification_report(y_predict,y_test,digits=5))

Instructions for updating:
Please use Model.predict, which supports generators.
              precision    recall  f1-score   support

           0    0.99247   0.95005   0.97080     10130
           1    0.00000   0.00000   0.00000         2
           2    0.00930   0.03279   0.01449        61
           3    0.00000   0.00000   0.00000         5
           4    0.00000   0.00000   0.00000         2
           5    0.00000   0.00000   0.00000         7

    accuracy                        0.94308     10207
   macro avg    0.16696   0.16381   0.16422     10207
weighted avg    0.98504   0.94308   0.96356     10207



In [64]:
#Tranfo mobile

seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)

"""
train_data_generator = ImageDataGenerator(
        #preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 0,
        zoom_range = 1.1,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip = False)
"""


train_data_generator = ImageDataGenerator()

test_data_generator = ImageDataGenerator()




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))

gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]


#y_train=train_generator.classes
#y_test=test_generator.classes


#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

datagen = ImageDataGenerator(
   )








val_accuracy=[]
accuracy=[]
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
#datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
#lenet.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
#                    steps_per_epoch=len(x_train) / 32, epochs=epochs)


"""
history=model.fit(x_train, y_train,validation_data=(x_test,y_test),verbose=0,
                 steps_per_epoch=1,
                           epochs=100,
                           workers=-1,
                           validation_steps=len(data_test))
"""

epochs=800

longueur=round(len(x_train)) 
for e in range(epochs):
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=600):
        history=model.fit(x_batch, y_batch,validation_data=(x_test,y_test),verbose=0)
        #val_accuracy.append(history.history["val_accuracy"])
        #accuracy.append(history.history["accuracy"])
        batches += 1
        if batches >= len(x_train) / longueur:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break 
            
            
            
y_predict=model.predict_generator(test_generator).argmax(axis=1)


Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


In [60]:

print(metrics.classification_report(y_predict,y_test,digits=5))

              precision    recall  f1-score   support

           0    0.99763   0.97168   0.98448      9956
           1    0.07407   0.66667   0.13333         3
           2    0.48372   0.90435   0.63030       115
           3    0.19403   0.92857   0.32099        14
           4    0.05714   1.00000   0.10811         2
           5    0.60241   0.85470   0.70671       117

    accuracy                        0.96943     10207
   macro avg    0.40150   0.88766   0.48065     10207
weighted avg    0.98575   0.96943   0.97597     10207



In [56]:
history.history["val_acc"]
WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 10207 batches). You may need to use the repeat() function when building your dataset.

[0.021945724]

In [63]:

print(metrics.classification_report(y_predict,y_test,digits=5))

              precision    recall  f1-score   support

           0    0.99588   0.97310   0.98435      9924
           1    0.14815   0.20000   0.17021        20
           2    0.52093   0.88889   0.65689       126
           3    0.07463   0.71429   0.13514         7
           4    0.34286   0.80000   0.48000        15
           5    0.56024   0.80870   0.66192       115

    accuracy                        0.96826     10207
   macro avg    0.44045   0.73083   0.51475     10207
weighted avg    0.98185   0.96826   0.97376     10207



In [47]:
batch_size

600

In [48]:
40827//600

68